In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import yaml

In [2]:
train_size = os.listdir("Dataset/train/images")
print(len(train_size))

15000


In [3]:
from ultralytics import YOLO

training_model = YOLO("yolov8n.yaml")


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
# train_images = os.listdir("Dataset/train/images")
# train_labels = os.listdir("Dataset/train/labels")
# REDUCED_SIZE = 15000
#
# print(len(os.listdir("Dataset/train/images")))
# for image, label in zip(train_images[REDUCED_SIZE:], train_labels[REDUCED_SIZE:]):
#     os.remove(os.path.join("Dataset/train/labels", label))
#     os.remove(os.path.join("Dataset/train/images", image))
# # print(os.listdir("Dataset/train/labels"))
# print(len(os.listdir("Dataset/train/labels")))

In [6]:
results = training_model.train(data="Dataset/data.yaml", epochs=50, device=0) # IMplement callback

New https://pypi.org/project/ultralytics/8.0.172 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.159  Python-3.11.0 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=Dataset/data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nm

In [7]:
results_new = training_model.predict("Dataset/train/images/000056694_jpg.rf.d3044032743b059a02077deb4bbf4cea.jpg")


image 1/1 C:\Users\Aman's Laptop\Documents\DNNProjects\PokerHandDetection\Dataset\train\images\000056694_jpg.rf.d3044032743b059a02077deb4bbf4cea.jpg: 640x640 1 2D, 2 8Cs, 1 9C, 20.0ms
Speed: 10.0ms preprocess, 20.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


In [17]:
import cv2

image = cv2.imread("test.jpg")
result = training_model.predict(image)

if image is not None:
    # Display the image in a window
    result_arr = result[0].boxes.data
    item_dict = result[0].names
    print(len(result_arr))
    for result in result_arr:
        cv2.rectangle(image, (int(result[0]), int(result[1])), (int(result[2]), int(result[3])), (120, int(result[5])/80 * 255, (1-int(result[5]))/80 * 255), 5)
    cv2.imshow('Image', image)

    # Wait for a key press and close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Image not found or couldn't be loaded.")


0: 384x640 1 8D, 1 9D, 21.0ms
Speed: 4.0ms preprocess, 21.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)


2


In [10]:
model = YOLO("best.pt")

In [25]:
# def display_text(text, frame, position_tuple):
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     font_scale = 1
#     font_color = (255, 255, 255)  # White color
#     font_thickness = 2
#     text_position = (position_tuple[0], position_tuple[1]-10) # Coordinates for text position
#     cv2.putText(frame, text, text_position, font, font_scale, font_color, font_thickness)
def display_text(text, frame, position):
    x, y = position
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    font_color = (255, 255, 255)  # White color
    font_thickness = 2
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    width, height = text_size
    cv2.rectangle(frame, (x , y-8 - height), ((x + 10 + width), y), (0, 0, 0), thickness = cv2.FILLED)
    cv2.putText(frame , text, (x+6, y -height + 4), font, font_scale, font_color, font_thickness)

In [33]:
import time
from HandClassification import detect_hand

cap = cv2.VideoCapture(0)
FPS = 45

while True:
    ret, frame = cap.read()
    start_time = time.time()
    if not ret:
        print("Error reading frame")
        break
    try:
        results = model.predict(frame)
        result_arr = results[0].boxes.data
        item_dict = results[0].names
        for result in result_arr:
            int_res = [int(x) for x in result]
            x1, y1, x2, y2, conf, label = int_res
            cv2.rectangle(frame, (x1, y1), (x2, y2),  (120, label/60 * 255, (1-label)/60 * 255), 5)
            display_text(item_dict[label], frame, (x1-10, y1-10))
        hand_des = [item_dict[int(result[5])] for result in result_arr]
        print(detect_hand(hand_des))
        if len(hand_des)  == 5:
            display_text(detect_hand(hand_des), frame, (100, 20))
    except Exception as e:
        print("Error occurred while making prediction", e)
    cv2.imshow("Live Camera Feed", frame)
    elapsed_time = time.time()-start_time
    overflow_time = max(0, 1/(FPS - elapsed_time))
    if cv2.waitKey(int(overflow_time*1000))& 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 15.0ms
Speed: 4.0ms preprocess, 15.0ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 8.8ms
Speed: 1.0ms preprocess, 8.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)



High Card 0 set()
0
High Card 0 set()
0


0: 480x640 (no detections), 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 13.1ms
Speed: 1.0ms preprocess, 13.1ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 15.0ms
Speed: 1.0ms preprocess, 15.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 12.0ms


High Card 0 set()
0
High Card 0 set()
0
High Card 0 set()
0


Speed: 2.9ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 9.0ms


...

Flush 5 {'D'}
5
Full House 6 {'D'}
6
Flush 5 {'D'}
5


Speed: 2.1ms preprocess, 13.0ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 3D, 13.0ms
Speed: 2.1ms preprocess, 13.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 3Ds, 12.0ms
Speed: 3.0ms preprocess, 12.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 13.0ms


Full House 6 {'D'}
6
Flush 5 {'D'}
5
Full House 6 {'D'}
6


Speed: 2.9ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 3D, 12.5ms
Speed: 1.9ms preprocess, 12.5ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)


High Card 0 set()
0
Flush 5 {'D'}
5
